In [1]:
# Load libraries
import numpy
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.tools.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
%matplotlib inline

/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
# Load full dataset
filename = 'ssdata_for_train_common.txt'
dataset_full = read_csv(filename, sep='\t')
dataset_full.shape

(1257780, 87)

In [3]:
# Select features to be used to build model
cols = ["Site.Type","TA.raw","SPS.raw","sRNA1A.scaled","sRNA1C.scaled","sRNA1G.scaled","sRNA8A.scaled","sRNA8C.scaled","sRNA8G.scaled","site8A.scaled","site8C.scaled","site8G.scaled","local.AU.raw","X3..pairing.raw","SA.raw","Min_dist.raw","PCT.raw","ORF.length.raw","X3.UTR.length.raw","Offset.6mer.raw","ORF.8mer.raw","transfection_final_conc_curated","trans_reagent","genetic_background","hours","GPL","Series","cell","log2fc"]
dataset_trim = dataset_full[cols]
dataset_trim.shape

(1257780, 29)

In [4]:
dataset_trim.head()

,Site.Type,TA.raw,SPS.raw,sRNA1A.scaled,sRNA1C.scaled,sRNA1G.scaled,sRNA8A.scaled,sRNA8C.scaled,sRNA8G.scaled,site8A.scaled,...,Offset.6mer.raw,ORF.8mer.raw,transfection_final_conc_curated,trans_reagent,genetic_background,hours,GPL,Series,cell,log2fc
0,7mer-m8,3.704,-8.80,0,0,0,0,0,1,0,...,1,0,2.500000e-08,Lipofectamine2000,wildtype,48.0,GPL570,GSE27431,HEY,0.167583
1,7mer-1a,3.621,-6.43,0,0,0,0,1,0,0,...,0,0,2.500000e-08,Lipofectamine2000,wildtype,48.0,GPL570,GSE27431,HEY,0.353088
2,6mer,3.539,-5.66,0,0,0,0,0,0,0,...,0,0,5.000000e-08,HiPerFect,wildtype,40.0,GPL4133,GSE28522,HeLa,0.275285
3,6mer,3.539,-5.66,0,0,0,0,0,0,0,...,0,0,5.000000e-08,HiPerFect,wildtype,40.0,GPL4133,GSE28522,TW01,-0.023325
4,6mer,3.569,-3.58,0,0,0,1,0,0,0,...,1,0,NaN,HiPerFect,wildtype,96.0,GPL13607,GSE29207,hMADS,0.009550


In [5]:
# Summarize Data

# shape
print(dataset_trim.shape)
# types
print(dataset_trim.dtypes)

(1257780, 29)
Site.Type                           object
TA.raw                             float64
SPS.raw                            float64
sRNA1A.scaled                        int64
sRNA1C.scaled                        int64
sRNA1G.scaled                        int64
sRNA8A.scaled                        int64
sRNA8C.scaled                        int64
sRNA8G.scaled                        int64
site8A.scaled                        int64
site8C.scaled                        int64
site8G.scaled                        int64
local.AU.raw                       float64
X3..pairing.raw                    float64
SA.raw                             float64
Min_dist.raw                       float64
PCT.raw                            float64
ORF.length.raw                     float64
X3.UTR.length.raw                  float64
Offset.6mer.raw                      int64
ORF.8mer.raw                         int64
transfection_final_conc_curated    float64
trans_reagent                       obje

### Preprocess Categorical Data

In [6]:
# Show classes in the features in object type, which must be transformed into digits.
obj_cols = ['Site.Type', "trans_reagent","genetic_background","GPL","Series","cell",]
for i, v in enumerate(obj_cols):
    print dataset_trim.groupby(v).size()
    print 

Site.Type
6mer       663452
7mer-1a    230616
7mer-m8    265919
8mer-1a     97793
dtype: int64

trans_reagent
Amaxa nucleofection             7284
Cell Line Nucleofector kit     30919
DharmaFECT                     22992
DharmaFECT3                     8502
HiPerFect                      66767
INTERFERin                      2575
Lentiviral vector              72953
Lipofectamin RNAi MAX         406776
Lipofectamine2000             378591
MicroPorator MP-100             2528
Neon transfection System        3642
Oligofectamine                 20673
PureFection                     6704
Retroviral vector              13148
SilentFect                     28828
Viral vector                    3042
X-tremeGENE siRNA               3857
siPORT neo-FX                  32412
dtype: int64

genetic_background
Dicer        119591
wildtype    1138189
dtype: int64

GPL
GPL10332    128884
GPL10558    127171
GPL10739      2972
GPL10904      7124
GPL13158      7466
GPL13497      8884
GPL13607    110402


In [7]:
dataset_enc = dataset_trim
mlb = []
obj_cols = ['Site.Type', "trans_reagent","genetic_background","GPL","Series","cell",]
for i, v in enumerate(obj_cols):    
    mlb.append(LabelEncoder())
    mlb[i].fit(dataset_trim[v])
    dataset_enc[v] = mlb[i].transform(dataset_trim[v])
    print "Number of classes for " + v + " : " + str(len(mlb[i].classes_))
    #print mlb[i].classes_
dataset_enc.head()

/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Number of classes for Site.Type : 4


/usr/local/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/usr/local/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:275: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


Number of classes for trans_reagent : 19
Number of classes for genetic_background : 2
Number of classes for GPL : 44
Number of classes for Series : 176
Number of classes for cell : 126


,Site.Type,TA.raw,SPS.raw,sRNA1A.scaled,sRNA1C.scaled,sRNA1G.scaled,sRNA8A.scaled,sRNA8C.scaled,sRNA8G.scaled,site8A.scaled,...,Offset.6mer.raw,ORF.8mer.raw,transfection_final_conc_curated,trans_reagent,genetic_background,hours,GPL,Series,cell,log2fc
0,2,3.704,-8.80,0,0,0,0,0,1,0,...,1,0,2.500000e-08,9,1,48.0,25,59,35,0.167583
1,1,3.621,-6.43,0,0,0,0,1,0,0,...,0,0,2.500000e-08,9,1,48.0,25,59,35,0.353088
2,0,3.539,-5.66,0,0,0,0,0,0,0,...,0,0,5.000000e-08,5,1,40.0,19,63,43,0.275285
3,0,3.539,-5.66,0,0,0,0,0,0,0,...,0,0,5.000000e-08,5,1,40.0,19,63,112,-0.023325
4,0,3.569,-3.58,0,0,0,1,0,0,0,...,1,0,NaN,5,1,96.0,6,66,121,0.009550


### Deal with NaN

In [8]:
dataset_unfilled = dataset_enc
dataset_unfilled.isnull().sum()

Site.Type                               0
TA.raw                                  0
SPS.raw                                 0
sRNA1A.scaled                           0
sRNA1C.scaled                           0
sRNA1G.scaled                           0
sRNA8A.scaled                           0
sRNA8C.scaled                           0
sRNA8G.scaled                           0
site8A.scaled                           0
site8C.scaled                           0
site8G.scaled                           0
local.AU.raw                            0
X3..pairing.raw                         0
SA.raw                                  0
Min_dist.raw                            0
PCT.raw                            239268
ORF.length.raw                          0
X3.UTR.length.raw                       0
Offset.6mer.raw                         0
ORF.8mer.raw                            0
transfection_final_conc_curated    361102
trans_reagent                           0
genetic_background                

In [12]:
# Imputing NaN with mean value of the entire column

imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(dataset_unfilled)
dataset = imr.transform(dataset_unfilled)

In [ ]:
seed = 7
numpy.random.seed(seed=seed)
rand_ix=numpy.random.randint(1, dataset.shape[0], size=30000)
rand_ix

In [45]:
# Prepare Data

# Split-out validation dataset

# Use Entire Data
#X = dataset[:,0:28]
#Y = dataset[:,28]

# Use only partial Data
X = dataset[rand_ix,0:28]
Y = dataset[rand_ix,28]

validation_size = 0.20
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [36]:
input_dim = 28

def plot_learning_curve(history):    
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.title('model loss - Mean Squared Error(MSE)')
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='upper right')
    pyplot.show()

# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

def more_larger_model():
	# create model
	model = Sequential()
	model.add(Dense(input_dim, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
	model.add(Dense(64, kernel_initializer='normal', activation='relu'))
	model.add(Dense(32, kernel_initializer='normal', activation='relu'))
	model.add(Dense(8, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
# Evaluate Algorithms
# Test options and evaluation metric
num_folds = 5
seed = 7
scoring = 'neg_mean_squared_error'

# Spot Check Algorithms
models = []
models.append(('LR', LinearRegression()))
models.append(('RIDGE', Ridge()))
models.append(('LASSO', Lasso()))
models.append(('EN', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('SVR', SVR()))
models.append(('XGB', XGBRegressor()))
models.append(('FFNN', KerasRegressor(build_fn=baseline_model, epochs=2000, batch_size=1024, verbose=0)))

# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure(figsize=(10, 7))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

LR: -0.612589 (0.045929)
RIDGE: -0.612588 (0.045929)
LASSO: -0.625626 (0.049576)
EN: -0.624634 (0.049577)
KNN: -0.432357 (0.018166)
CART: -0.800002 (0.041370)


In [ ]:
# Test options and evaluation metric
num_folds = 3
seed = 7
scoring = 'neg_mean_squared_error'

# Standardize the dataset
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
pipelines.append(('ScaledRIDGE', Pipeline([('Scaler', StandardScaler()),('RIDGE', Ridge())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledSVR', Pipeline([('Scaler', StandardScaler()),('SVR', SVR())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB', XGBRegressor())])))
pipelines.append(('ScaledFFNN', Pipeline([('Scaler', StandardScaler()),('FFNN', KerasRegressor(build_fn=baseline_model, epochs=2000, batch_size=1024, verbose=0))])))

results = []
names = []

for name, model in pipelines:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure(figsize=(10, 7))
fig.suptitle('Scaled Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [43]:
# grid search
max_depth = range(1, 11, 2)
print(max_depth)
param_grid = dict(max_depth=max_depth)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train, Y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    pyplot.errorbar(max_depth, means, yerr=stds)
    pyplot.title("XGBoost max_depth vs MSE")
    pyplot.xlabel('max_depth')
    pyplot.ylabel('Log Loss')
    pyplot.savefig('max_depth.png')

[1, 3, 5, 7, 9]
Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/local/anaconda2/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

...................................

### Tuning Feed Forward Neural Network Topologies

In [ ]:
# create model


history = []

epochs=1000
batch_size=1024

# Fit the model
model = baseline_model()
his = model.fit(X_train, Y_train, validation_data=(X_validation,Y_validation), epochs=epochs, batch_size=batch_size, verbose=0)
history.append(his)

model = larger_model()
his = model.fit(X_train, Y_train, validation_data=(X_validation,Y_validation), epochs=epochs, batch_size=batch_size, verbose=0)
history.append(his)

#model = wider_model()
#his = model.fit(X_train, Y_train, validation_data=(X_validation,Y_validation), epochs=epochs, batch_size=batch_size, verbose=0)
#history.append(his)

#model = more_larger_model()
#his = model.fit(X_train, Y_train, validation_data=(X_validation,Y_validation), epochs=epochs, batch_size=batch_size, verbose=0)
#history.append(his)

plot_learning_curve(history[0])
plot_learning_curve(history[1])
plot_learning_curve(history[2])
plot_learning_curve(history[3])

In [ ]:
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop' , 'adam']
inits = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Deep Neural Network Algorithm tuning
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
k_values = numpy.array([1,3,5,7,9,11,13,15,17,19,21])
param_grid = dict(n_neighbors=k_values)
model = KNeighborsRegressor()
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# ensembles
ensembles = []
ensembles.append(('ScaledAB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostRegressor())])))
ensembles.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))
ensembles.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor())])))
ensembles.append(('ScaledET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor())])))
results = []
names = []
for name, model in ensembles:
	kfold = KFold(n_splits=num_folds, random_state=seed)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

# Compare Algorithms
fig = pyplot.figure()
fig.suptitle('Scaled Ensemble Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
# Tune scaled GBM
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=numpy.array([50,100,150,200,250,300,350,400]))
model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, Y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Make predictions on validation dataset

# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = GradientBoostingRegressor(random_state=seed, n_estimators=400)
model.fit(rescaledX, Y_train)
# transform the validation dataset
rescaledValidationX = scaler.transform(X_validation)
predictions = model.predict(rescaledValidationX)
print(mean_squared_error(Y_validation, predictions))